In [60]:
#Import relevant libraries for object image analysis we will be comparing
#Single Shot Detector(SSD) model vs. Faster-RCNN
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, ssd300_vgg16
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pandas as pd
import time
import os

In [61]:
# Load COCO class labels dynamically
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
COCO_CLASSES = weights.meta["categories"]

# Load models
faster_rcnn = fasterrcnn_resnet50_fpn(weights=weights)
faster_rcnn.eval()
ssd = ssd300_vgg16(pretrained=True)
ssd.eval()

# Image transformation
transform = T.Compose([T.ToTensor()])

# Load images
image_folder = "/content/drive/MyDrive/MSBA 503/images"  # Update with your path
images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.jpg', '.png'))]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [62]:
# Visualization function using Matplotlib
def visualize_results(image, boxes, labels, scores, title):
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    for box, score, label in zip(boxes, scores, labels):
        x_min, y_min, x_max, y_max = box
        rect = patches.Rectangle(
            (x_min, y_min),
            x_max - x_min,
            y_max - y_min,
            linewidth=2,
            edgecolor='red',
            facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(
            x_min,
            y_min - 5,
            f"{label}: {score:.2f}",
            color='white',
            fontsize=10,
            bbox=dict(facecolor='red', alpha=0.5)
        )
    ax.set_title(title)
    plt.axis("off")
    plt.show()

In [63]:
# Process results with commented-out visualization
results = []

for image_path in images:
    img_name = os.path.basename(image_path)
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image)

    # Measure detection time for Faster R-CNN
    start_time = time.time()
    with torch.no_grad():
        faster_rcnn_preds = faster_rcnn([image_tensor])[0]
    faster_rcnn_time = time.time() - start_time

    # Extract Faster R-CNN predictions
    faster_rcnn_boxes = faster_rcnn_preds['boxes'].tolist()
    faster_rcnn_scores = faster_rcnn_preds['scores'].tolist()
    faster_rcnn_labels = [COCO_CLASSES[label] for label in faster_rcnn_preds['labels'].tolist()]
    faster_rcnn_filtered = [(b, l, s) for b, l, s in zip(faster_rcnn_boxes, faster_rcnn_labels, faster_rcnn_scores) if s > 0.5]

    # Measure detection time for SSD
    start_time = time.time()
    with torch.no_grad():
        ssd_preds = ssd([image_tensor])[0]
    ssd_time = time.time() - start_time

    # Extract SSD predictions
    ssd_boxes = ssd_preds['boxes'].tolist()
    ssd_scores = ssd_preds['scores'].tolist()
    ssd_labels = [COCO_CLASSES[label] for label in ssd_preds['labels'].tolist()]
    ssd_filtered = [(b, l, s) for b, l, s in zip(ssd_boxes, ssd_labels, ssd_scores) if s > 0.5]

    # Visualization (commented out)
    # if faster_rcnn_filtered:
    #     boxes, labels, scores = zip(*faster_rcnn_filtered)
    #     visualize_results(image, boxes, labels, scores, title=f"Faster R-CNN: {img_name}")
    # if ssd_filtered:
    #     boxes, labels, scores = zip(*ssd_filtered)
    #     visualize_results(image, boxes, labels, scores, title=f"SSD: {img_name}")

    # Extract additional insights
    dimensions = image.size  # Image dimensions (width, height)
    top_left_color = image.getpixel((0, 0))  # Dominant color at top-left pixel

    # Record results for comparison
    results.append({
        'Image': img_name,
        'Faster R-CNN Time (s)': round(faster_rcnn_time, 4),
        'Faster R-CNN Objects': len(faster_rcnn_filtered),
        'Faster R-CNN Avg Confidence': round(sum([s for _, _, s in faster_rcnn_filtered]) / len(faster_rcnn_filtered), 4) if faster_rcnn_filtered else 0,
        'SSD Time (s)': round(ssd_time, 4),
        'SSD Objects': len(ssd_filtered),
        'SSD Avg Confidence': round(sum([s for _, _, s in ssd_filtered]) / len(ssd_filtered), 4) if ssd_filtered else 0,
        'Image Dimensions': f"{dimensions[0]}x{dimensions[1]}",
        'Top-Left Color': str(top_left_color),
    })

# Save results to DataFrame
df = pd.DataFrame(results)

# Round all numerical columns to 4 decimal places
df = df.round(4)

# Preview results
print("Detection Results (Preview):")
print(df.head())

Detection Results (Preview):
                                               Image  Faster R-CNN Time (s)  \
0  hide-the-pain-harold-why-you-should-not-use-st...                 9.2922   
1              Stock-Photography-vs-Real-Imagery.jpg                 8.5109   
2       premium_photo-1661508620175-3ae20da61cda.jpg                 9.5575   
3                                     cat_caviar.jpg                 9.3356   
4       premium_photo-1661772661721-b16346fe5b0f.jpg                 7.7109   

   Faster R-CNN Objects  Faster R-CNN Avg Confidence  SSD Time (s)  \
0                     9                       0.8003        2.1429   
1                    13                       0.7102        1.2645   
2                     6                       0.8656        1.4605   
3                     7                       0.7923        1.2556   
4                    15                       0.7823        2.1482   

   SSD Objects  SSD Avg Confidence Image Dimensions   Top-Left Color  
0   